In [1]:
import os
os.environ['PYSPARK_PYTHON'] = '/nfshome/jc9033/.conda/envs/py3/bin/python3.7'

from pyspark import SparkContext
sc = SparkContext('local', 'pyspark')

from pyspark.sql.session import SparkSession
spark = SparkSession(sc)

In [2]:
sc

<SparkContext master=local appName=pyspark>

## fhv

In [3]:
#  Generate a year-month list with a start year-month and an end year-month
def generate_year_month_list(start, end):
    monthlist = []
    for year in range(int(start[:4]),int(end[:4])+1):
        for month in range(1,13):
            if month >= 10:
                temp = str(year) + "-" + str(month) 
            else:
                temp = str(year) + "-" + '0' + str(month) 
            monthlist.append(temp)
    if int(end[-2:]) == 12:
        return monthlist[int(start[-2:])-1:]
    else:
        return monthlist[int(start[-2:])-1: int(end[-2:])-12]

In [4]:
year_month_list = generate_year_month_list("2015-01","2018-06")

# check columns for each month
for i in year_month_list:    
    path = "data_fhv/fhv_" + i + ".csv"
    temp = sc.textFile(path).cache()
    print(i, list(enumerate(temp.first().split(','))))

In [5]:
# 2015-01 : 2016-12, Pickup_date & locationID
# 2017-01 : 2017-06, Pickup_DateTime & DropOff_datetime & PUlocationID & DOlocationID
# 2017-07 : 2018-06, Pickup_DateTime & DropOff_datetime & PUlocationID & DOlocationID & SR_Flag

### Processing functions

In [6]:
def outlier_delete(x):
    try:
        if int(x) in range(1,264):
            return x
        else:
            return "0"
    except:
        return "0"

In [7]:
UberLyft_base_num = ['B02512','B02395','B02617','B02682','B02764','B02765','B02835','B02836','B02864','B02865','B02866','B02867','B02869','B02870','B02871','B02872','B02875','B02876','B02877','B02878','B02879','B02880','B02882','B02883','B02884','B02887','B02888','B02889']

In [8]:
def mapper1_1(partitionID, rows):
    if partitionID==0:
        next(rows)
    import csv
    reader = csv.reader(rows)
    for fields in reader:
        if fields[0] in UberLyft_base_num:
            yield((fields[1][:10], fields[2]),1)

def mapper1_2(partitionID, rows):
    if partitionID==0:
        next(rows)
    import csv
    reader = csv.reader(rows)
    for fields in reader:
        if fields[0] in UberLyft_base_num:
            yield((fields[1][:10], fields[3], fields[4]),1)
        
def mapper2_1(partition):
    for element in partition:
        yield(element[0][0], outlier_delete(element[0][1]), "0", element[1])
        
def mapper2_2(partition):
    for element in partition:
        yield(element[0][0], outlier_delete(element[0][1]), outlier_delete(element[0][2]), element[1])

# Cleaning & Integrating

In [ ]:
%%time
for i in year_month_list:
    print(i)
    path = "data_fhv/fhv_" + i + ".csv"
    if i == "2015-01":
        rdd = sc.textFile(path)\
                 .mapPartitionsWithIndex(mapper1_1)\
                 .groupByKey().mapValues(sum)\
                 .mapPartitions(mapper2_1)
        rdd.take(5)
    else:
        if int(i[:4])<2017:
            temp = sc.textFile(path)\
                     .mapPartitionsWithIndex(mapper1_1)\
                     .groupByKey().mapValues(sum)\
                     .mapPartitions(mapper2_1)
        else:
            temp = sc.textFile(path)\
                     .mapPartitionsWithIndex(mapper1_2)\
                     .groupByKey().mapValues(sum)\
                     .mapPartitions(mapper2_2)
        rdd = rdd.union(temp)
        rdd.take(5)

2015-01
2015-02
2015-03
2015-04
2015-05
2015-06
2015-07
2015-08
2015-09
2015-10
2015-11
2015-12
2016-01
2016-02
2016-03
2016-04
2016-05
2016-06
2016-07
2016-08
2016-09
2016-10
2016-11
2016-12
2017-01
2017-02
2017-03
2017-04
2017-05
2017-06
2017-07


### Output CSV

In [ ]:
%%time
from pyspark import SparkContext
df = spark.createDataFrame(rdd, ['date', 'PUlocationID', 'DOlocationID', 'count'])

In [ ]:
df.show()

In [ ]:
%%time
df.coalesce(1).write.format('com.databricks.spark.csv').options(header='true').save('output_UberLyft')

###  Check file

In [ ]:
filename = ! ls output_UberLyft | grep .csv

In [ ]:
import pandas as pd
pddf = pd.read_csv('output_UberLyft/'+filename[0]).fillna(0)
print(pddf.shape)

In [ ]:
pddf.head()

In [ ]:
pddf.tail()